In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from datetime import datetime
import scipy.optimize
import datashader as ds
import colorcet as cc
from sklearn.cluster import KMeans, MiniBatchKMeans
import holoviews as hv
from holoviews import opts
from holoviews.element.tiles import StamenTerrain
import holoviews.operation.datashader as hd
import statsmodels.formula.api as smf

In [157]:
N = 400
T = 5
sigma = .0003
noise  = np.random.normal(0,sigma,(N,T))
alpha_1 = .2
alpha_2 = .12
ybar = 0

In [160]:
def data_generator(N, T, sigma, ybar, alpha_1, alpha_2):
    arr = np.random.normal(0,1,(2,N)) 

    for t in range(1,T+1):
        y_t = arr[-1:,:] * alpha_1 + arr[-2:-1,:] * alpha_2 #+  np.random.normal(ybar,0,(1,N))
        arr = np.concatenate((arr, y_t), axis = 0)
    
    df = pd.DataFrame()
    df['user'] = list(np.arange(1,N+1))*(T+2)
    df['metric'] = np.concatenate(arr)
    df.sort_values(by=['user'], inplace=True)
    df['time'] = df.groupby('user').cumcount()
    df['time'] = df['time'] - 2
    df['lag_1'] = df.groupby('user')['metric'].shift(1)
    df['lag_2'] = df.groupby('user')['metric'].shift(2)
    #df['time'] = df.groupby('user').count
    return(df)
    

In [161]:
df = data_generator(N, T, sigma, 1, alpha_1, alpha_2)
df

,user,metric,time,lag_1,lag_2
0,1,0.187581,-2,NaN,NaN
2000,1,0.056121,-1,0.187581,NaN
800,1,0.383437,0,0.056121,0.187581
1200,1,0.293243,1,0.383437,0.056121
1600,1,0.104661,2,0.293243,0.383437
...,...,...,...,...,...
1199,400,-0.279848,0,-0.330574,-0.038729
1999,400,-0.073597,1,-0.279848,-0.330574
799,400,-1.200896,2,-0.073597,-0.279848
1599,400,-0.200077,3,-1.200896,-0.073597


In [162]:
formula='metric ~  lag_1 + lag_2 - 1'

result1 =  smf.ols(formula=formula, data=df).fit()
print(result1.summary())

                                 OLS Regression Results                                
Dep. Variable:                 metric   R-squared (uncentered):                   0.040
Model:                            OLS   Adj. R-squared (uncentered):              0.039
Method:                 Least Squares   F-statistic:                              41.56
Date:                Fri, 14 Oct 2022   Prob (F-statistic):                    2.07e-18
Time:                        18:10:41   Log-Likelihood:                         -1592.6
No. Observations:                2000   AIC:                                      3189.
Df Residuals:                    1998   BIC:                                      3200.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [100]:
np.concatenate(arr)

array([-1.23055211,  1.48999845, -0.80934699,  0.24699584, -1.87252446,
       -0.09530446, -1.1041545 ,  0.59464965, -0.05708783, -0.64326994,
       -0.31303167,  1.54315687, -0.40644634,  0.46817727, -0.82005526,
       -0.34816443,  1.08382827, -0.03497284,  0.38867756,  0.55141604,
       -0.21027259,  0.48743119, -0.17841091,  0.12327495, -0.38871399,
       -0.08106942,  0.08426711,  0.06436339,  0.07088497,  0.03309081,
       -0.07961832,  0.28266506, -0.08445574,  0.08083626, -0.17614943,
       -0.05799362,  0.14691282,  0.00867594,  0.0608183 ,  0.07278809])

In [58]:
np.arange(1,4, )

array([1, 2, 3])

In [63]:
np.concatenate(np.arange(1,5), axis=1)

ValueError: zero-dimensional arrays cannot be concatenated